In [7]:
import os

import sagemaker
from sagemaker.pytorch import PyTorch

## Upload Data

In [3]:
session = sagemaker.Session()

bucket = session.default_bucket()
s3_prefix = 'capstone/v1'

role = sagemaker.get_execution_role()

In [4]:
data_dir = 'processed'
input_data = session.upload_data(path=data_dir, bucket=bucket, key_prefix=s3_prefix)

## Train the Model

In [15]:
output_path = f's3://{bucket}/{s3_prefix}/output'

estimator = PyTorch(
    source_dir='src',
    entry_point='train.py',
    role=role,
    framework_version='1.1.0',
    py_version='py3',
    train_instance_count=1,
    train_instance_type='ml.p2.xlarge',
    output_path=output_path,
    hyperparameters={
        'seed': 1,
        'batch-size': 512,
        'epochs': 2,
        'embedding-dim': 32,
        'num-lstm-layers': 1,
        'hidden-dims': '100',
        # vocab size from previous step + 2 for
        # out of vocab and empty
        'vocab-size': 10002
    }
)

train_instance_count has been renamed in sagemaker>=2.
See: https://sagemaker.readthedocs.io/en/stable/v2.html for details.
train_instance_type has been renamed in sagemaker>=2.
See: https://sagemaker.readthedocs.io/en/stable/v2.html for details.


In [16]:
estimator.fit({
    'train': os.path.join(input_data, 'test'),
    'eval': os.path.join(input_data, 'val')
})

2021-03-21 19:58:04 Starting - Starting the training job...
2021-03-21 19:58:07 Starting - Launching requested ML instancesProfilerReport-1616356684: InProgress
.........
2021-03-21 20:00:00 Starting - Preparing the instances for training......
2021-03-21 20:01:01 Downloading - Downloading input data...
2021-03-21 20:01:21 Training - Downloading the training image...
2021-03-21 20:02:01 Training - Training image download completed. Training in progress.bash: cannot set terminal process group (-1): Inappropriate ioctl for device
bash: no job control in this shell
2021-03-21 20:01:51,811 sagemaker-containers INFO     Imported framework sagemaker_pytorch_container.training
2021-03-21 20:01:51,836 sagemaker_pytorch_container.training INFO     Block until all host DNS lookups succeed.
2021-03-21 20:01:51,838 sagemaker_pytorch_container.training INFO     Invoking user training script.
2021-03-21 20:01:52,108 sagemaker-containers INFO     Module train does not provide a setup.py. 
Generating 

UnexpectedStatusException: Error for Training job sagemaker-pytorch-2021-03-21-19-58-04-199: Failed. Reason: AlgorithmError: ExecuteUserScriptError:
Command "/usr/bin/python -m train --batch-size 512 --embedding-dim 32 --epochs 2 --hidden-dims 100 --num-lstm-layers 1 --seed 1 --vocab-size 10002"
[nltk_data] Downloading collection 'all'
[nltk_data]    | 
[nltk_data]    | Downloading package abc to /root/nltk_data...
[nltk_data]    |   Unzipping corpora/abc.zip.
[nltk_data]    | Downloading package alpino to /root/nltk_data...
[nltk_data]    |   Unzipping corpora/alpino.zip.
[nltk_data]    | Downloading package biocreative_ppi to
[nltk_data]    |     /root/nltk_data...
[nltk_data]    |   Unzipping corpora/biocreative_ppi.zip.
[nltk_data]    | Downloading package brown to /root/nltk_data...
[nltk_data]    |   Unzipping corpora/brown.zip.
[nltk_data]    | Downloading package brown_tei to /root/nltk_data...
[nltk_data]    |   Unzipping corpora/brown_tei.zip.
[nltk_data]    | Downloading pac

## Next Steps

- Get the model training successfully on SageMaker
- Get the model predicting successfully
- Determine the metric we'll use for eval and hyperparameter tuning
- Do full run, evaluate results:

- Create hyperparameter tuning job